In [47]:
import pandas as pd
import sklearn
import imblearn
import os

data_dir = 'D:\FINKI\8_dps\Project\DATA\physionet_dbs\cudb\clean_segments_ft_250hz'
write_data_dir = 'D:\FINKI\8_dps\Project\DATA\MODELS_DATA\\base_models_all_dft_coefficients'
if not os.path.exists(write_data_dir):
    os.makedirs(write_data_dir)

In [48]:
def undersampling_dict_generation(y_):

    class_feature = pd.Series(y_)
    return {
        class_label: class_feature.value_counts().min()
        for class_label in list(class_feature.value_counts().to_dict().keys())
    }

In [51]:
best_pbc_corrs_by_dataset = dict()

for segments_length_seconds in [3,4,5,6,7,8,9,10]:
    file = f'ft_segments_{segments_length_seconds}s.csv'; print(file)

    dataset = pd.read_csv(f'{data_dir}\\{file}')

    dataset = dataset[dataset['episode'].isin(['N', 'VF'])]
    dataset.replace({'N': 0, 'VF': 1}, inplace = True)

    dataset['segment_fft'] = dataset['segment_fft'].apply(lambda seg_list: eval(seg_list))

    for dft_coefficient_iter in range(len(dataset.iloc[0]['segment_fft'])):
        dataset[f'dft_coefficient_{dft_coefficient_iter}'] = dataset['segment_fft'].apply(
            lambda ft_segment: ft_segment[dft_coefficient_iter]
        )

    dataset.drop(labels = ['segment_fft'], axis = 1, inplace = True)

    # knn undersampler did not accept dictionary as sampling_strategy even though
    # in the docs it was said it does, thus opting for random undersampling for initial models
    random_undersampler = imblearn.under_sampling.RandomUnderSampler(
            sampling_strategy = undersampling_dict_generation
    )

    y = dataset.pop('episode')
    X = dataset

    X, y = random_undersampler.fit_resample(X,y)

    shuffle_splitter = sklearn.model_selection.ShuffleSplit(test_size=.1, random_state=42)
    train_idxs, test_idxs = next(shuffle_splitter.split(X))

    X_train = X.iloc[train_idxs]
    y_train = y.iloc[train_idxs]
    X_test = X.iloc[test_idxs]
    y_test = y.iloc[test_idxs]

    standard_scaler = sklearn.preprocessing.StandardScaler()

    X_train = pd.DataFrame(standard_scaler.fit_transform(X_train), columns = X_train.columns)
    X_test = pd.DataFrame(standard_scaler.fit_transform(X_test), columns = X_test.columns)

    X_train['episode'] = y_train.reset_index(drop = True)
    X_test['episode'] = y_test.reset_index(drop = True)

    train = X_train
    test = X_test

    train.to_csv(f'{write_data_dir}\\segments_{segments_length_seconds}s_train.csv', index = False)
    test.to_csv(f'{write_data_dir}\\segments_{segments_length_seconds}s_test.csv', index = False)


ft_segments_3s.csv
ft_segments_4s.csv
ft_segments_5s.csv
ft_segments_6s.csv
ft_segments_7s.csv
ft_segments_8s.csv
ft_segments_9s.csv
ft_segments_10s.csv
